# Data extraction, analysis and visualisation for IMBES conference 

This code was put together to share with my supervisor to extract EEG data to present at a conference this summer. 

Participants completed  ~120 trials. This trial takes the EEG associated with each trial and averages accross multiple experimental conditions. It then visualises the data to select the key time points for analysis.

## Setup file

Import files and set up custom functions

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from scipy.signal import find_peaks
import pingouin as pg

import mne

# Function to update event codes
def update_events(raw):
    events, labels = mne.events_from_annotations(raw, verbose='Warning')

    target_onset_trigger = labels['Stimulus/target_onset']
    trigger_1 = labels['Stimulus/S  1'] if 'Stimulus/S  1' in labels else '199'
    trigger_2 = labels['Stimulus/S  2'] if 'Stimulus/S  2' in labels else '299'
    trigger_3 = labels['Stimulus/S  3'] if 'Stimulus/S  3' in labels else '399'
    trigger_5 = labels['Stimulus/S  5'] if 'Stimulus/S  5' in labels else '599'
    trigger_6 = labels['Stimulus/S  6'] if 'Stimulus/S  6' in labels else '699'
    trigger_7 = labels['Stimulus/S  7'] if 'Stimulus/S  7' in labels else '799'

    print(trigger_1, trigger_2, trigger_3, trigger_5, trigger_6, trigger_7)

    num_trig = [trigger_1, trigger_2, trigger_3, trigger_5, trigger_6, trigger_7]

    for i in range(len(events)):
        if events[i, 2] == target_onset_trigger:
            j = i
            while True:
                j += 1
                if events[j, 2] in num_trig:
                    if events[j, 2] in [trigger_1, trigger_7]:
                        events[i, 2] = 300
                    elif events[j, 2] in [trigger_2, trigger_6]:
                        events[i, 2] = 200
                    elif events[j, 2] in [trigger_3, trigger_5]:
                        events[i, 2] = 100
                    break

    if 100 in events[:,2]:
        labels['Stimulus/target_close'] = 100
    if 200 in events[:,2]:    
        labels['Stimulus/target_mid'] = 200
    if 300 in events[:,2]:
        labels['Stimulus/target_far'] = 300

    return events, labels



    # Function to detect local peaks using find_peaks
def detect_peaks(evoked, tmin_n1=None, tmax_n1=None, tmin_p2=None, tmax_p2=None, tmin_p3b=None, tmax_p3b=None, peak_to_detect=None):
    times = evoked.times
    data = evoked.data[0, :]
    
    if peak_to_detect == 'N1':
        # Detect local minima (N1) within the specified time range
        idx_n1_window = np.where((times >= tmin_n1) & (times <= tmax_n1))[0]
        data_n1_window = data[idx_n1_window]
        n1_peaks, _ = find_peaks(-data_n1_window)  # Negate data to find minima
        if n1_peaks.size > 0:
            n1_peak_idx = idx_n1_window[n1_peaks[0]]
            n1_latency = times[n1_peak_idx]
            n1_amplitude = data[n1_peak_idx]
        else:
            n1_latency, n1_amplitude = np.nan, np.nan

        return n1_latency, n1_amplitude

    elif peak_to_detect == 'P2':
        # Detect local maxima (P2) within the specified time range
        idx_p2_window = np.where((times >= tmin_p2) & (times <= tmax_p2))[0]
        data_p2_window = data[idx_p2_window]
        p2_peaks, _ = find_peaks(data_p2_window)
        if p2_peaks.size > 0:
            p2_peak_idx = idx_p2_window[p2_peaks[0]]
            p2_latency = times[p2_peak_idx]
            p2_amplitude = data[p2_peak_idx]
        else:
            p2_latency, p2_amplitude = np.nan, np.nan

        return p2_latency, p2_amplitude

    else:
        # Detect both N1 and P2 peaks
        # Detect local minima (N1) within the specified time range
        idx_n1_window = np.where((times >= tmin_n1) & (times <= tmax_n1))[0]
        data_n1_window = data[idx_n1_window]
        n1_peaks, _ = find_peaks(-data_n1_window)  # Negate data to find minima
        if n1_peaks.size > 0:
            n1_peak_idx = idx_n1_window[n1_peaks[0]]
            n1_latency = times[n1_peak_idx]
            n1_amplitude = data[n1_peak_idx]
        else:
            n1_latency, n1_amplitude = np.nan, np.nan

        # Detect local maxima (P2) within the specified time range
        idx_p2_window = np.where((times >= tmin_p2) & (times <= tmax_p2))[0]
        data_p2_window = data[idx_p2_window]
        p2_peaks, _ = find_peaks(data_p2_window)
        if p2_peaks.size > 0:
            p2_peak_idx = idx_p2_window[p2_peaks[0]]
            p2_latency = times[p2_peak_idx]
            p2_amplitude = data[p2_peak_idx]
        else:
            p2_latency, p2_amplitude = np.nan, np.nan

        # Detect local maxima (P2) within the specified time range
        idx_p3b_window = np.where((times >= tmin_p3b) & (times <= tmax_p3b))[0]
        data_p3b_window = data[idx_p3b_window]
        p3b_peaks, _ = find_peaks(data_p3b_window)
        if p3b_peaks.size > 0:
            p3b_peak_idx = idx_p3b_window[p3b_peaks[0]]
            p3b_latency = times[p3b_peak_idx]
            p3b_amplitude = data[p3b_peak_idx]
        else:
            p3b_latency, p3b_amplitude = np.nan, np.nan
        
        return n1_latency, n1_amplitude, p2_latency, p2_amplitude, p3b_latency, p3b_amplitude





## Extract Grand averages from ppts

In [ ]:
# Load your preprocessed EEG data for each subject and calculate grand average
subject_files = glob.glob('cleaned_seg_child/*.vhdr')  # Get all EEG files in the 'cleaned_seg_child' folder (update this path as needed)
skip = []  # List of participant IDs to skip
run = []   # (Optional) List of specific participants to run, currently not used

# Initialize lists to store participant data and results
PID = []  
overall_evoked = []  
close_evoked = []  
mid_evoked = []  
far_evoked = []  
overall_evoked_P8 = [] 
close_evoked_P8 = [] 
mid_evoked_P8 = []  
far_evoked_P8 = []  
peaks_list = []  #
mean_amp_list = [] 
evoked_df = pd.DataFrame()  
seg_length = pd.DataFrame()  
no_p8 = []  

# Loop through all EEG files
for file in subject_files:
    # Extract participant ID from the filename by removing specific parts of the string
    id = file.replace('cleaned_seg_child/', "").replace('_test_Artifact Rejection 1.vhdr', "").replace('_Test_Artifact Rejection 1.vhdr', "").replace('_baseline_Artifact Rejection 1.vhdr', "")

    # Skip files if the participant ID is in the 'skip' list
    if id in skip:
        print(f'Skipping {file} as it was in the skip subject list')
        continue

    else:
        try:
            PID.append(id)  # Store the participant ID
            print(f'Processing participant: {id}')

            # Load the raw EEG data using the BrainVision format
            raw = mne.io.read_raw_brainvision(file, preload=True, verbose='Warning')
            # Optionally, apply a filter (commented out)
            # raw = raw.filter(l_freq=0.1, h_freq=30, verbose='Warning')

            # Extract events and labels from the raw data (assuming a custom function 'update_events')
            events, labels = update_events(raw)

            # Set the montage (electrode positions) for the EEG data
            montage = mne.channels.make_standard_montage("easycap-M1")
            raw.set_montage(montage, verbose='Warning')

            # Create an empty event dictionary to map condition names to event codes
            event_dict = {}
            if 'Stimulus/target_close' in labels:
                event_dict['target_close'] = labels['Stimulus/target_close']
            if 'Stimulus/target_mid' in labels:
                event_dict['target_mid'] = labels['Stimulus/target_mid']
            if 'Stimulus/target_far' in labels:
                event_dict['target_far'] = labels['Stimulus/target_far']

            # Create epochs (time-locked segments) based on the events with a baseline correction
            epochs = mne.Epochs(raw, events, event_id=event_dict, tmin=-0.2, tmax=0.5, preload=True, baseline=(-0.2, 0), verbose='Warning')

            # Check if the channel 'P8' is present in the data
            if 'P8' in epochs.info['ch_names']:
                overall_evoked_P8.append(epochs.average())  # Append average evoked response for all conditions at P8
            else:
                no_p8.append(id)  # Store participant ID if P8 is missing

            overall_evoked.append(epochs.average())  # Append the overall evoked response for all conditions

            # Initialize a dictionary to store the number of epochs for each condition
            evoked_dict = {'PID': id}
            
            # For 'close' condition
            if 'Stimulus/target_close' in labels:
                close_evoked.append(epochs['target_close'].average())  # Append average evoked response for 'close' condition
                if 'P8' in epochs.info['ch_names']:
                    close_evoked_P8.append(epochs['target_close'].average())  # Append evoked response at P8 for 'close' condition
                evoked_dict['close'] = len(epochs['target_close'])  # Store the number of epochs in 'close' condition
            else:
                evoked_dict['close'] = 0  # If no 'close' condition, set it to 0

            # For 'mid' condition
            if 'Stimulus/target_mid' in labels:
                mid_evoked.append(epochs['target_mid'].average())  # Append average evoked response for 'mid' condition
                if 'P8' in epochs.info['ch_names']:
                    mid_evoked_P8.append(epochs['target_mid'].average())  # Append evoked response at P8 for 'mid' condition
                evoked_dict['mid'] = len(epochs['target_mid'])  # Store the number of epochs in 'mid' condition
            else:
                evoked_dict['mid'] = 0  # If no 'mid' condition, set it to 0

            # For 'far' condition
            if 'Stimulus/target_far' in labels:
                far_evoked.append(epochs['target_far'].average())  # Append average evoked response for 'far' condition
                if 'P8' in epochs.info['ch_names']:
                    far_evoked_P8.append(epochs['target_far'].average())  # Append evoked response at P8 for 'far' condition
                evoked_dict['far'] = len(epochs['target_far'])  # Store the number of epochs in 'far' condition
            else:
                evoked_dict['far'] = 0  # If no 'far' condition, set it to 0

            # Convert evoked data dictionary to a DataFrame and concatenate with the overall segment length DataFrame
            evoked_dict = pd.DataFrame(evoked_dict, index=[0])
            seg_length = pd.concat([seg_length, evoked_dict], ignore_index=True)

        except Exception as error:
            # Handle exceptions, raise them to stop the script, and print the error and filename
            raise
            print("An exception occurred:", error)
            print(file)

# Calculate the grand average evoked responses for each condition across participants
overall_grand_average = mne.grand_average(overall_evoked)
close_grand_average = mne.grand_average(close_evoked)
mid_grand_average = mne.grand_average(mid_evoked)
far_grand_average = mne.grand_average(far_evoked)

# Calculate the grand average evoked responses for each condition across participants at P8 channel
overall_grand_average_P8 = mne.grand_average(overall_evoked_P8)
close_grand_average_P8 = mne.grand_average(close_evoked_P8)
mid_grand_average_P8 = mne.grand_average(mid_evoked_P8)
far_grand_average_P8 = mne.grand_average(far_evoked_P8)




## ERP and Topo maps joint

In [ ]:

topo_grand_average = overall_grand_average_P8.copy()
topo_grand_average.drop_channels(['Fp1', 'Fp2'])
overall_topo = topo_grand_average.plot_topomap([-0.2, 0, 0.15, 0.25, 0.375, 0.5])


### P3 and P4 grand averages

In [ ]:
# Set time windows to extact ERPs
tmin_n1 = 0.13
tmax_n1 = 0.22
tmin_p2 = 0.21
tmax_p2 = 0.28

%matplotlib inline

# Copy grand average data frames for each channel;
p3_grand_average = overall_grand_average.copy()
p4_grand_average = overall_grand_average.copy()

# get timesa and data for P3 and P4 channels
p3_times = p3_grand_average.pick(['P3']).times
p3_data = p3_grand_average.pick(['P3']).data[0]

p4_times = p4_grand_average.pick(['P4']).times
p4_data = p4_grand_average.pick(['P4']).data[0]


# Plot the P3 and P4 ERPs
plt.figure(figsize=(10, 5))
plt.plot(p3_times, p3_data, label='P3 Grand Average')
plt.plot(p4_times, p4_data, label='P4Grand Average')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (µV)')
plt.legend()
# Mark the N1 region
plt.axvspan(tmin_n1, tmax_n1, color='blue', alpha=0.1, label='N1 Region')


# Mark the P2 region
plt.axvspan(tmin_p2, tmax_p2, color='red', alpha=0.1, label='P2 Region')
# Mark the detected P2 peak
plt.show()

In [ ]:
# Set time windows to extact ERPs
tmin_n1 = 0.13
tmax_n1 = 0.22
tmin_p2 = 0.21
tmax_p2 = 0.28

%matplotlib inline

# Copy grand average data frames for each channel
p7_grand_average = overall_grand_average_P8.copy()
p8_grand_average = overall_grand_average_P8.copy()

# get timesa and data for P7 and P8 channels
p7_times = p7_grand_average.pick(['P7']).times
p7_data = p7_grand_average.pick(['P7']).data[0]

p8_times = p8_grand_average.pick(['P8']).times
p8_data = p8_grand_average.pick(['P8']).data[0]


plt.figure(figsize=(10, 5))
plt.plot(p7_times, p7_data, label='P3 Grand Average')
plt.plot(p8_times, p8_data, label='P4Grand Average')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (µV)')
plt.legend()
# Mark the N1 region
plt.axvspan(tmin_n1, tmax_n1, color='blue', alpha=0.1, label='N1 Region')


# Mark the P2 region
plt.axvspan(tmin_p2, tmax_p2, color='red', alpha=0.1, label='P2 Region')
# Mark the detected P2 peak
plt.show()

## Global Field Power plots

In [ ]:
overall_gfp = overall_grand_average.plot(gfp=True, spatial_colors=True, ylim=dict(eeg=[-12, 12]))

## Evoked plots for selcted elctrodes

In [ ]:
left_picks = ["P3", "P7", "O1"]
right_picks = ["P4", "P8", "O2"]

overall_left_plot = overall_grand_average.plot(picks=left_picks)
overall_right_plot = overall_grand_average.plot(picks=right_picks)

## Detect peaks using a local minima/maxima from scipy

In [ ]:
# Load your preprocessed EEG data for each subject and calculate grand average
subject_files = glob.glob('cleaned_seg_child/*.vhdr')  # Update with your actual file path
skip = []# Initialize an empty list to store peak data for each participant and condition
run = []

# Define time windows for N1, P2, and P3b components
tmin_n1 = 0.13
tmax_n1 = 0.22
tmin_p2 = 0.21
tmax_p2 = 0.28
tmin_p3b = 0.25
tmax_p3b = 0.5

peaks_list = []  # List to store detected peak data

# Loop through each EEG file (participant)
for file in subject_files:

    # Extract participant ID from the filename
    id = file.replace('cleaned_seg_child/', "").replace('_test_Artifact Rejection 1.vhdr', "").replace('_Test_Artifact Rejection 1.vhdr', "").replace('_baseline_Artifact Rejection 1.vhdr', "")

    # Skip participants in the skip list
    if id in skip:
        print(f'Skipping {file} as it was in the skip subject list')
        continue  

    else:
        PID.append(id)  # Store participant ID

        # Load the raw EEG data
        print(f'Processing participant: {id}')
        raw = mne.io.read_raw_brainvision(file, preload=True, verbose='Warning')
        events, labels = update_events(raw)

        # Set montage (electrode placement) for the EEG data
        montage = mne.channels.make_standard_montage("easycap-M1")
        raw.set_montage(montage, verbose='Warning')

        # Create an event dictionary for different stimulus conditions
        event_dict = {}
        if 'Stimulus/target_close' in labels:
            event_dict['target_close'] = labels['Stimulus/target_close']
        if 'Stimulus/target_mid' in labels:
            event_dict['target_mid'] = labels['Stimulus/target_mid']
        if 'Stimulus/target_far' in labels:
            event_dict['target_far'] = labels['Stimulus/target_far']

        # Create epochs for the EEG data for each event
        epochs = mne.Epochs(raw, events, event_id=event_dict, tmin=-0.2, tmax=0.7, preload=True, baseline=(None, 0), verbose='Warning')

        # Create evoked responses for each condition
        evoked_dict = {}
        if 'Stimulus/target_close' in labels:
            evoked_dict['close'] = epochs['target_close']
        if 'Stimulus/target_mid' in labels:
            evoked_dict['mid'] = epochs['target_mid']
        if 'Stimulus/target_far' in labels:
            evoked_dict['far'] = epochs['target_far']

        # Loop through each condition and channel to detect peaks
        for condition, evoked in evoked_dict.items():
            for channel in ['P3', 'P4', 'P7', 'P8']:
                
                # Skip specific case for participant 117 and channel P8
                if (id == '117' and channel == 'P8'):
                    continue

                # Set different time windows for central (P3, P4) vs lateral (P7, P8) channels
                if channel in ['P3', 'P4']:
                    tmin_n1, tmax_n1 = 0.12, 0.18
                    tmin_p2, tmax_p2 = 0.18, 0.25
                else:
                    tmin_n1, tmax_n1 = 0.13, 0.19
                    tmin_p2, tmax_p2 = 0.19, 0.25
                
                # Get the evoked response for the selected channel and detect peaks
                evoked_channel = evoked.copy().average().pick([channel])
                n1_latency, n1_amplitude, p2_latency, p2_amplitude, p3b_latency, p3b_amplitude = detect_peaks(
                    evoked_channel, tmin_n1=tmin_n1, tmax_n1=tmax_n1, tmin_p2=tmin_p2, tmax_p2=tmax_p2, tmin_p3b=tmin_p3b, tmax_p3b=tmax_p3b)

                # Store detected peak data in the list
                peaks_list.append({
                    'PID': id,
                    'Condition': condition,
                    'Channel': channel,
                    'N1_Latency': n1_latency,
                    'N1_Amplitude': n1_amplitude,
                    'P2_Latency': p2_latency,
                    'P2_Amplitude': p2_amplitude,
                    'P3b_Latency': p3b_latency,
                    'P3b_Amplitude': p3b_amplitude
                })

# Convert the list of peaks to a DataFrame for further analysis
original_peaks_df = pd.DataFrame(peaks_list)
# Save the DataFrame to a CSV file if needed
# original_peaks_df.to_csv('child_subjects_peaks.csv')





## Plot comparisons for each condition P3 and P4

In [ ]:
import mne

# Define the electrode picks for left (P3) and right (P4) hemispheres
right_picks = ['P4']
left_picks = ['P3']

# Pick left hemisphere channel (P3) and combine across conditions using the average
left_ix = mne.pick_channels(overall_grand_average.info["ch_names"], include=left_picks)
overall_roi_left = mne.channels.combine_channels(overall_grand_average, {'left_ROI':left_ix}, method="mean")
close_roi_left = mne.channels.combine_channels(close_grand_average, {'left_ROI':left_ix}, method="mean")
mid_roi_left = mne.channels.combine_channels(mid_grand_average, {'left_ROI':left_ix}, method="mean")
far_roi_left = mne.channels.combine_channels(far_grand_average, {'left_ROI':left_ix}, method="mean")

# Pick right hemisphere channel (P4) and combine across conditions using the average
right_ix = mne.pick_channels(overall_grand_average.info["ch_names"], include=right_picks)
overall_roi_right = mne.channels.combine_channels(overall_grand_average, {'right_ROI':right_ix}, method="mean")
close_roi_right = mne.channels.combine_channels(close_grand_average, {'right_ROI':right_ix}, method="mean")
mid_roi_right = mne.channels.combine_channels(mid_grand_average, {'right_ROI':right_ix}, method="mean")
far_roi_right = mne.channels.combine_channels(far_grand_average, {'right_ROI':right_ix}, method="mean")

# Combine evoked responses for left hemisphere across conditions for comparison
full_left_grand_averages = dict(close=close_roi_left, mid=mid_roi_left, far=far_roi_left)
full_left_compare_plot = mne.viz.plot_compare_evokeds(full_left_grand_averages)

# Combine evoked responses for right hemisphere across conditions for comparison
full_right_grand_averages = dict(close=close_roi_right, mid=mid_roi_right, far=far_roi_right)
full_right_compare_plot = mne.viz.plot_compare_evokeds(full_right_grand_averages)

# Plot a subset of conditions (close and mid) for left hemisphere
left_grand_averages = dict(close=close_roi_left, mid=mid_roi_left)
left_compare_plot = mne.viz.plot_compare_evokeds(left_grand_averages)

# Plot a subset of conditions (close and mid) for right hemisphere
right_grand_averages = dict(close=close_roi_right, mid=mid_roi_right)
right_compare_plot = mne.viz.plot_compare_evokeds(right_grand_averages)




### Comparison plots for P7 and P8

In [ ]:
import mne

# Define the electrode picks for left (P8) and right (P7) hemispheres
right_picks = ['P7']
left_picks = ['P8']

# Pick left hemisphere channels (P8) and combine across conditions using the mean
left_ix = mne.pick_channels(overall_grand_average_P8.info["ch_names"], include=left_picks)
overall_roi_left = mne.channels.combine_channels(overall_grand_average_P8, {'left_ROI':left_ix}, method="mean")
close_roi_left = mne.channels.combine_channels(close_grand_average_P8, {'left_ROI':left_ix}, method="mean")
mid_roi_left = mne.channels.combine_channels(mid_grand_average_P8, {'left_ROI':left_ix}, method="mean")
far_roi_left = mne.channels.combine_channels(far_grand_average_P8, {'left_ROI':left_ix}, method="mean")

# Pick right hemisphere channels (P7) and combine across conditions using the mean
right_ix = mne.pick_channels(overall_grand_average_P8.info["ch_names"], include=right_picks)
overall_roi_right = mne.channels.combine_channels(overall_grand_average_P8, {'right_ROI':right_ix}, method="mean")
close_roi_right = mne.channels.combine_channels(close_grand_average_P8, {'right_ROI':right_ix}, method="mean")
mid_roi_right = mne.channels.combine_channels(mid_grand_average_P8, {'right_ROI':right_ix}, method="mean")
far_roi_right = mne.channels.combine_channels(far_grand_average_P8, {'right_ROI':right_ix}, method="mean")

# Combine evoked responses for left hemisphere across all conditions and plot them
full_left_grand_averages = dict(close=close_roi_left, mid=mid_roi_left, far=far_roi_left)
full_left_compare_plot = mne.viz.plot_compare_evokeds(full_left_grand_averages)

# Combine evoked responses for right hemisphere across all conditions and plot them
full_right_grand_averages = dict(close=close_roi_right, mid=mid_roi_right, far=far_roi_right)
full_right_compare_plot = mne.viz.plot_compare_evokeds(full_right_grand_averages)

# Plot a subset of conditions (close and mid) for left hemisphere
left_grand_averages = dict(close=close_roi_left, mid=mid_roi_left)
left_compare_plot = mne.viz.plot_compare_evokeds(left_grand_averages)

# Plot a subset of conditions (close and mid) for right hemisphere
right_grand_averages = dict(close=close_roi_right, mid=mid_roi_right)
right_compare_plot = mne.viz.plot_compare_evokeds(right_grand_averages)


## ANOVA removing Far condition

In [ ]:
peaks_df = original_peaks_df[original_peaks_df['Condition'] != 'far']
peaks_df = peaks_df.loc[peaks_df['Channel'].isin(['P3', 'P4'])]

# Add Hemisphere column based on Channel
peaks_df['Hemisphere'] = peaks_df['Channel'].map({'P3': 'Left', 'P4': 'Right'})


# Perform ANOVA for N1 Latency
anova_n1_latency = pg.rm_anova(dv='N1_Latency', within=['Condition', 'Hemisphere'], subject='PID', data=peaks_df)
print("ANOVA Results for N1 Latency:")
print(anova_n1_latency)

# Perform ANOVA for N1 Amplitude
anova_n1_amplitude = pg.rm_anova(dv='N1_Amplitude', within=['Condition', 'Hemisphere'], subject='PID', data=peaks_df)
print("ANOVA Results for N1 Amplitude:")
print(anova_n1_amplitude)

# Perform ANOVA for P2 Latency
anova_p2_latency = pg.rm_anova(dv='P2_Latency', within=['Condition', 'Hemisphere'], subject='PID', data=peaks_df)
print("ANOVA Results for P2 Latency:")
print(anova_p2_latency)

# Perform ANOVA for P2 Amplitude
anova_p2_amplitude = pg.rm_anova(dv='P2_Amplitude', within=['Condition', 'Hemisphere'], subject='PID', data=peaks_df)
print("ANOVA Results for P2 Amplitude:")
print(anova_p2_amplitude)

